In [81]:
import pandas as pd
import datetime
import os
from tqdm import tqdm
import codecs
import re
import string
import spacy
import es_core_news_sm
from datetime import datetime, timedelta
import tldextract

In [79]:
#%pip install google.cloud

In [519]:
#%pip install tldextract

In [2]:
# Dataset 1 --> representación de entrada única, tipo tabla
personal_info = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/personal_info.csv')

In [3]:
personal_info['Created on'] = personal_info['Created on'].apply(pd.to_datetime)

In [4]:
for i in range(len(personal_info['phone'])):
    personal_info['phone'][i] = personal_info['phone'][i].replace('+', '00')

<ipython-input-4-2740362463fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_info['phone'][i] = personal_info['phone'][i].replace('+', '00')


In [5]:
personal_info[['phone_', 'country']] = personal_info['phone'].str.split('[', 1, expand=True)

In [6]:
personal_info.drop(columns=['phone'], inplace=True)

In [7]:
for i in range(len(personal_info['country'])):
    personal_info['country'][i] = personal_info['country'][i].replace(']', '')

<ipython-input-7-820afdb3e02d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  personal_info['country'][i] = personal_info['country'][i].replace(']', '')


In [8]:
personal_info.head(1)

,Google Account ID,Created on,IP,Language,e-mail,recovery mail,name,gender,phone_,country
0,907204290308,2017-07-20 08:00:00,213.4.207.110,es,jrdiazordaz03@gmail.com,IronhideX1@gmail.com,Jose Diaz,male,0034615686348,ES


In [63]:
personal_info.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/npl/input/dataset_1a.csv', index=False)

In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [9]:
# Dataset 2 --> representación de gráficos, tradicionales
activity = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/activity.csv')

In [10]:
for i in range(len(activity['date'])):
    activity['date'][i] = activity['date'][i].replace('UTC', '')
    activity['date'][i].strip()

In [11]:
activity['date'] = activity['date'].apply(pd.to_datetime)

In [12]:
activity.head(1)

,ip,activity,date
0,2.136.118.161,Login,2021-12-06 10:27:15


In [84]:
activity.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/npl/input/dataset_2a.csv', index=False)

In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [127]:
# Dataset 3 --> modelo predictivo sobre las canciones
music = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/df_music_comb_final.csv')

In [128]:
music.loc[0]

url                                 music.youtube.com/watch?v=6xzN8Nt0Pok
title                          I Wanna Dance with Somebody (Who Loves Me)
album_x                                                           Whitney
artist                                                    Whitney Houston
publishedAt                                          2017-02-18T18:29:59Z
channelId                                        UCcoxwwIh_afLwJHVBEQw62Q
description             Provided to YouTube by Arista/Legacy\n\nI Wann...
categoryId                                                             10
channelTitle                                     UCcoxwwIh_afLwJHVBEQw62Q
liveBroadcastContent                                                 none
ids                                                           6xzN8Nt0Pok
album_y                 {'album_type': 'album', 'artists': [{'external...
artists                 [{'external_urls': {'spotify': 'https://open.s...
available_markets       ['AD', 'AE', '

In [130]:
music.drop(columns=['description', 'album_y', 'artists', 'is_local', 'publishedAt', 'channelTitle', 'url',
                   'external_urls', 'href', 'preview_url', 'id', 'reference', 'uri', 'type', 'external_ids',
                   'ids'], inplace=True)

In [131]:
music.rename(columns={'album_x':'album', 'channelId':'channel', 'liveBroadcastContent':'islive',
                     'available_markets':'countries'}, inplace=True)

In [132]:
music.head(1)

,title,album,artist,channel,categoryId,islive,countries,disc_number,duration_ms,explicit,popularity,track_number,lyrics
0,I Wanna Dance with Somebody (Who Loves Me),Whitney,Whitney Houston,UCcoxwwIh_afLwJHVBEQw62Q,10,none,"['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT...",1,291293,False,83,1,huh yeah woo hey yeah huh ooh yeah uh huh yea...


In [133]:
music.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/npl/input/dataset_3a.csv', index=False)

In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [15]:
# Dataset 2 --> representación de gráficos, tradicionales
history = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/history.csv')

In [16]:
history.head(1)

,page_transition,title,url,client_id,time_usec
0,LINK,Horarios de autobús 151: Madrid (Plaza Castill...,https://www.redtransporte.com/madrid/autobuses...,x6cQo08w9vTcM7buu4Mugg==,1638781091719312


In [17]:
history.drop(columns=['client_id'],inplace=True)

In [18]:
history['time_usec2']= pd.to_datetime(history['time_usec'],unit='us',utc=True)
history['PST_time'] = history['time_usec2'].dt.tz_convert('America/Vancouver')
history['month'] = history['PST_time'].dt.month
history['hour'] = history['PST_time'].dt.hour
history['day_of_week'] = history['PST_time'].dt.day_name()
history['weekday'] = history['PST_time'] \
                    .apply(lambda x: "Y" if x.weekday()<5 else "N")

In [19]:
# Parse URL to get domain information
def get_domain(x):
    if x.startswith('chrome-extension://'):
        return "chrome_extension"
    domain = tldextract.extract(x)[1]
    sub_domain = tldextract.extract(x)[0]
    if domain == "google":
        if sub_domain=="www":
            return "google_search"
        else:
            return sub_domain + "." + domain
    return domain
history['domain'] = history['url'].apply(get_domain)

In [20]:
history.drop(columns=['time_usec','PST_time'],inplace=True)

In [27]:
history_date = []
history_time = []
for x in history['time_usec2']:
    history_date.append(str(x).split(' ')[0])
    history_time.append(str(x).split(' ')[1])

In [56]:
history['date'] = history_date
history['time'] = history_time

In [57]:
temp_ = []
for x in history['time']:
    temp_.append(str(x).split('+')[0])

In [58]:
history['time'] = temp_

In [74]:
history.drop(columns=['time_usec2','url'],inplace=True)

In [75]:
history.head(1)

,page_transition,title,month,hour,day_of_week,weekday,domain,date,time
0,LINK,Horarios de autobús 151: Madrid (Plaza Castill...,12,0,Monday,Y,redtransporte,2021-12-06,08:58:11.719312


In [76]:
history.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/npl/input/dataset_2b.csv', index=False)

In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [225]:
# Dataset 2 --> representación de gráficos, tradicionales
favorites = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/favs.csv')

In [226]:
favorites.head(1)

,description
0,Canciones Simpson 07x04 Jardín del Eden


In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [270]:
# Dataset 3 --> modelo predictivo agrupado sobre contenido de emails
mail = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/mail_2.csv')

In [271]:
mail.drop(columns=(['labels','Subject','thread']), inplace=True)

In [274]:
mail.head(1)

,from,to,Time,date,body,quarter
0,provomad@cruzroja.es,jrdiazordaz03@gmail.com,00:57:23,2021-12-03,estimado jose ramon: los próximos días 14...,FA


In [235]:
mail.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/npl/input/dataset_3b.csv', index=False)

In [ ]:
////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

## NPL

In [16]:
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.corpus import wordnet
import re
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from textblob import TextBlob

[nltk_data] Downloading package punkt to /home/krujo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/krujo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/krujo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [109]:
lyrics_npl = pd.read_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/output/npl_favorites.csv')

In [110]:
lyrics_npl.drop(columns=(['description', 'url']), inplace=True)

In [111]:
lyrics_npl.rename(columns={'date2':'date', 'desc':'lyrics'}, inplace=True)

In [112]:
lyrics_npl.head(1)

,date,lyrics
0,2020-01-13 21:55:10,¿Sabes lo que te digo? - YouTube


In [113]:
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    return text

# Cleaning the text in the review column
lyrics_npl['lyrics'] = lyrics_npl['lyrics'].apply(clean)
lyrics_npl.head()

,date,lyrics
0,2020-01-13 21:55:10,Sabes lo que te digo YouTube
1,2019-07-05 08:57:19,Bruce Springsteen Secret Garden HD Jerry Magui...
2,2020-02-15 19:19:28,Average White Band Pick up the pieces YouTube
3,2019-11-21 14:40:31,Def Leppard Love Bites YouTube
4,2019-09-18 11:58:47,The Matrix Clubbed to Death hours YouTube


In [114]:
# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
            newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

lyrics_npl['lyrics_tagged'] = lyrics_npl['lyrics'].apply(token_stop_pos)
lyrics_npl.head(2)

,date,lyrics,lyrics_tagged
0,2020-01-13 21:55:10,Sabes lo que te digo YouTube,"[(Sabes, n), (lo, n), (que, n), (te, n), (digo..."
1,2019-07-05 08:57:19,Bruce Springsteen Secret Garden HD Jerry Magui...,"[(Bruce, n), (Springsteen, n), (Secret, n), (G..."


In [115]:
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

lyrics_npl['lyrics_lemma'] = lyrics_npl['lyrics_tagged'].apply(lemmatize)
lyrics_npl.head(1)

,date,lyrics,lyrics_tagged,lyrics_lemma
0,2020-01-13 21:55:10,Sabes lo que te digo YouTube,"[(Sabes, n), (lo, n), (que, n), (te, n), (digo...",Sabes lo que te digo YouTube


In [116]:
# function to calculate subjectivity
def getSubjectivity(review):
    return TextBlob(review).sentiment.subjectivity
    # function to calculate polarity
    def getPolarity(review):
        return TextBlob(review).sentiment.polarity

# function to analyze the reviews
def analysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [117]:
fin_data = pd.DataFrame(lyrics_npl[['date','lyrics', 'lyrics_lemma']])

In [118]:
# fin_data['Subjectivity'] = fin_data['Lemma'].apply(getSubjectivity) 
fin_data['Polarity'] = fin_data['lyrics_lemma'].apply(getSubjectivity) 
fin_data['Analysis'] = fin_data['Polarity'].apply(analysis)
fin_data.head()

,date,lyrics,lyrics_lemma,Polarity,Analysis
0,2020-01-13 21:55:10,Sabes lo que te digo YouTube,Sabes lo que te digo YouTube,0.0,Neutral
1,2019-07-05 08:57:19,Bruce Springsteen Secret Garden HD Jerry Magui...,Bruce Springsteen Secret Garden HD Jerry Mag...,0.7,Positive
2,2020-02-15 19:19:28,Average White Band Pick up the pieces YouTube,Average White Band Pick piece YouTube,0.2,Positive
3,2019-11-21 14:40:31,Def Leppard Love Bites YouTube,Def Leppard Love Bites YouTube,0.6,Positive
4,2019-09-18 11:58:47,The Matrix Clubbed to Death hours YouTube,Matrix Clubbed Death hour YouTube,0.0,Neutral


In [119]:
fin_data.to_csv('/mnt/c/Ironhack/Material/Repositorio/Final/data/test/tableau/2.favorites.csv', index=False)

In [120]:
tb_counts = fin_data.Analysis.value_counts()

tb_counts

Neutral     67
Positive    33
Name: Analysis, dtype: int64